In [5]:
import os
from tranformers import DetrImageProcessor, DetrForObjectDetection
import torch
from torch.utils.data import Dataset
from PIL import Image
import pandas as pd
import numpy as np
import pandas as pd
from torchvision.io import read_image
from torchvision.transforms import functional as tv_F
from pathlib import Path
from transformers import Trainer, TrainingArguments, AutoImageProcessor, AutoModelForObjectDetection
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
import numpy as np
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")



In [6]:
df = pd.read_csv('data.csv')
df

,img_id,cam_id,seq_id,frame_id,bbox_id,x_min,x_max,y_min,y_max,label_l1,label_l2,fao_mfa,train,val,test
0,da7c5ef0-db28-11ea-b693-9b3523cc10a5,60,13209,33,1,626,754,-1,88,Human,HUMAN,81,True,False,False
1,da7c5ef0-db28-11ea-b693-9b3523cc10a5,60,13209,33,3,544,710,15,203,Human,HUMAN,81,True,False,False
2,da7c5ef0-db28-11ea-b693-9b3523cc10a5,60,13209,33,4,629,735,210,562,Human,HUMAN,81,True,False,False
3,da7c5ef0-db28-11ea-b693-9b3523cc10a5,60,13209,33,5,615,883,550,907,Human,HUMAN,81,True,False,False
4,da7c5ef0-db28-11ea-b693-9b3523cc10a5,60,13209,33,6,885,962,826,875,Unknown,OTH,81,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549204,fcdecc18-5952-11ec-a87d-47788f674eed,93,15,28,126812,227,510,702,992,Yellowfin tuna,YFT,81,False,False,True
549205,fcdeef2c-5952-11ec-a8ae-cf526681fdda,93,15,77,128339,223,518,700,996,Yellowfin tuna,YFT,81,False,False,True
549206,fcdeef2c-5952-11ec-a8ae-cf526681fdda,93,15,77,128338,875,1127,668,1030,Yellowfin tuna,YFT,81,False,False,True
549207,fcdeef2c-5952-11ec-a8ae-cf526681fdda,93,15,77,128337,1069,1465,487,674,Yellowfin tuna,YFT,81,False,False,True


In [7]:
df.columns

Index(['img_id', 'cam_id', 'seq_id', 'frame_id', 'bbox_id', 'x_min', 'x_max',
       'y_min', 'y_max', 'label_l1', 'label_l2', 'fao_mfa', 'train', 'val',
       'test'],
      dtype='object')

In [8]:
image_dir = Path("img")
label_file = "data.csv"

def load_labels(label_path,train=True):
    df = pd.read_csv(label_path)
    annotations = []
    if train:
        df = df[df['train']==True].reset_index()
    else:
        df = df[df['val']==True].reset_index()
    df=df[:1000]# убрать 
    for img_id, group in df.groupby("img_id"):
        boxes = group[["x_min", "y_min", "x_max", "y_max"]].values.tolist()
        labels = group["label_l1"].tolist()
        annotations.append({"image_id": img_id, "boxes": boxes, "labels": labels})
    return annotations
annotations = load_labels(label_file)
annotations_test = load_labels(label_file,False)[:1000] # havent_resurses
def load_dataset(annotations):
    dataset = []
    for annotation in annotations:
        image_path = image_dir / f"{annotation['image_id']}.jpg"
        # image = read_image(str(image_path))
        # image = tv_F.convert_image_dtype(image, dtype=torch.float32)
        dataset.append({"image_path": image_path, "annotations": annotation})
    return dataset

In [9]:
dataset = {
    "train": load_dataset(annotations),
    "test": load_dataset(annotations_test)
}

In [10]:
def collate_fn(examples):
    def load_img(path):
        image = read_image(str(path))
        image = tv_F.convert_image_dtype(image, dtype=torch.float32)
        return image
    images = [load_img(example["image_path"]) for example in examples]
    pixel_values = processor(images=images, return_tensors="pt")['pixel_values']
    labels = [example["annotations"] for example in examples]
    return {"pixel_values": pixel_values, "labels": labels}


In [11]:
from evaluate import load_metric

In [13]:
metric = load_metric("detection")

def compute_metrics(p):
    predictions, labels = p
    
    results = metric.compute(predictions, labels,processor, iou=0.5 )
    return {
        "precision": results["precision"],
        "recall": results["recall"],
        "f1": results["f1"],
        "mAP50": results["mAP50"],
        "mAP50-90": results["mAP50-90"],
    }


In [22]:


# training_dataloader = DataLoader(dataset["train"], batch_size=1, shuffle=True)



# training_args = TrainingArguments(
#     output_dir="./results",
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     num_train_epochs=3,
#     fp16=True,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     logging_dir="./logs",
#     learning_rate=2e-5,
#     weight_decay=0.01,
#     load_best_model_at_end=True,
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=dataset["train"],
#     eval_dataset=dataset["test"],
#     data_collator=collate_fn,
#     tokenizer=processor,

# )


# trainer.train()
# trainer.save_model("./best")
'asdsdasdsad'

Steps,Training Loss,Validation Loss,Precision_box,Recall_box,F1_box,MAP50_box
5000,No log,0.365465,0.732505,0.357975,0.479091,0.414750
10000,No log,0.253454,0.873148,0.468065,0.592857,0.426129


TrainOutput(global_step=846, training_loss=0.19623384783344885, metrics={'train_runtime': 26789.81, 'train_samples_per_second': 127.464, 'train_steps_per_second': 1.027, 'total_flos': 1593623665686096.0, 'train_loss': 0.19623384783344885, 'epoch': 10.0})